# Very quick presentation of Theano

**Theano** is a compiler for mathematical expressions written in Theano. Devise any mathematical expression in the syntax of the well-established core numerics package **numpy**, and Theano will create C-code to evaluate this expression and compile it. It is also possible to generate cuda-code for GPU by simply changing a flag. Of great interest is also the fact that gradients for any scalar function can be calculated automatically, making gradient errors in the construction and training of neural networks a thing of the past.

### Structure of this notebook
- a micro-presentation of numpy
- a micro-introduction to Theano
- a logistic regression with Theano
- an XOR network with Theano
- a multi-layer perceptron with Theano